XG boost is an great model due to its consitentcy, efficency, and reliablity which makes this an impotant AI model to try out. Numeric features were tested but non numeric features were used as they gave better results. All outliers where LOS is over 21 days were dropped.

In [1]:
%pip install seaborn
%pip install missingno
%pip install plotly
%pip install numpy
%pip install pandas
%pip install nltk
%pip install nbformat
%pip install scikit-learn
%pip install jupyter tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

pip install --upgrade --force-reinstall xgboost


  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scipy-1.16.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached xgboost-3.0.2-py3-none-win_amd64.whl (150.0 MB)
Using cached numpy-2.3.2-cp312-cp312-win_amd64.whl (12.8 MB)
Using cached scipy-1.16.1-cp312-cp312-win_amd64.whl (38.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Read Data

In [4]:
df = pd.read_csv('hospital_cleaned.csv')
df.drop(df[df['length_of_stay'] > 21].index, inplace=True)

## Convert applicable colunms to numeric

In [5]:
# age group
age_code_map = {
    "0 to 17":      0,
    "18 to 29":     1,
    "30 to 49":     2,
    "50 to 69":     3,
    "70 or Older":  4
}

# Map the age group to a numerical code
df["age_group_code"] = df["age_group"].map(age_code_map)

# Quick check of the result
print(df[["age_group", "age_group_code"]].head())

     age_group  age_group_code
1     18 to 29               1
2  70 or Older               4
3     50 to 69               3
4     50 to 69               3
5      0 to 17               0


In [6]:
mortality_risk_code_map = {
   "Minor":      0,
   "Moderate":     1,
   "Major":     2,
   "Extreme":     3,
}
df["apr_risk_of_mortality_code"] = df["apr_risk_of_mortality"].map(mortality_risk_code_map)
# 4. Quick check
print(df[["apr_risk_of_mortality", "apr_risk_of_mortality_code"]].head())

  apr_risk_of_mortality  apr_risk_of_mortality_code
1                 Minor                           0
2                 Major                           2
3                 Minor                           0
4              Moderate                           1
5                 Minor                           0


In [7]:
illness_severity_code_map = {
   "Minor":      0,
   "Moderate":     1,
   "Major":     2,
   "Extreme":     3,
}
df["apr_severity_of_illness_code"] = df["apr_severity_of_illness_description"].map(illness_severity_code_map)
# 4. Quick check
print(df[["apr_severity_of_illness_description", "apr_severity_of_illness_code"]].head())

  apr_severity_of_illness_description  apr_severity_of_illness_code
1                               Minor                             0
2                               Major                             2
3                            Moderate                             1
4                               Major                             2
5                            Moderate                             1


## Features 

In [8]:
## all features non numeirc
features = [
   "gender",
    "type_of_admission",
    "apr_medical_surgical_description",
    "emergency_department_indicator",
    "ccsr_diagnosis_description",
    "ccsr_procedure_description",
    "apr_drg_description",
    "apr_mdc_description",
    "age_group",
    "apr_risk_of_mortality",
    "apr_severity_of_illness_description"
    
]


# feature split between numeric and categorical
categorical_features = [
    "gender",
    "type_of_admission",
    "apr_medical_surgical_description",
    "emergency_department_indicator",
    "ccsr_diagnosis_description",
    "ccsr_procedure_description",
    "apr_drg_description",
    "apr_mdc_description"
]
numeric_features = [
    "age_group_code",
    "apr_severity_of_illness_code",
    "apr_risk_of_mortality_code"
]
# define target variable
target = "length_of_stay"
# Change x value to categorical + numeric featrues to use numeric
X = df[features]
y = df[target]

## One hot encode

In [ ]:
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), features) # <-- change this features to categorical features if you are using them
], remainder='drop')
X_enc = preprocessor.fit_transform(X)

: 

## Create, train and test the model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42
)
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    early_stopping_rounds=10,   
    eval_metric='rmse'
)
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=True
)

## Get the results

In [ ]:
preds = xgb_model.predict(X_test)
mse  = mean_squared_error(y_test, preds)
rmse = mse**0.5
mae  = mean_absolute_error(y_test, preds)
print(f"Test MSE : {mse:.3f}")
print(f"Test RMSE: {rmse:.3f}")
print(f"Test MAE : {mae:.3f}")

Test MSE : 9.779
Test RMSE: 3.127
Test MAE : 2.103
